In [1]:
#this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in the Shared Google Drive
FOLDERNAME = 'Shared drives/CS 230 Project'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that

# python files from within it.
import sys
sys.path.append('/content/drive/{}'.format(FOLDERNAME))

%cd /content/drive/$FOLDERNAME/

Mounted at /content/drive
/content/drive/Shared drives/CS 230 Project


In [3]:
!pip install sentence-transformers

     |████████████████████████████████| 81kB 4.4MB/s 
     |████████████████████████████████| 2.3MB 11.0MB/s 
     |████████████████████████████████| 1.2MB 44.3MB/s 
     |████████████████████████████████| 3.3MB 23.1MB/s 
     |████████████████████████████████| 901kB 38.7MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.2.0-cp37-none-any.whl size=123339 sha256=18cff8391b708be3a0f25f6aa4823065baf5e56e165c1a36a676f661d799111f
  Stored in directory: /root/.cache/pip/wheels/0f/06/f7/faaa96fdda87462b4fd5c47b343340e9d5531ef70d0eef8242
Successfully built sentence-transformers


In [4]:
from sentence_transformers import SentenceTransformer, models, InputExample, losses, SentencesDataset
from torch import nn
from torch.utils.data import DataLoader
import pickle



In [ ]:
word_embedding_model = models.Transformer('bert-base-uncased', max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=256, activation_function=nn.Tanh())
lstm_model = models.LSTM(word_embedding_dimension=300, hidden_dim=300, num_layers=1, dropout=0)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model, lstm_model])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:

model = SentenceTransformer('distilbert-base-nli-mean-tokens')
train_examples = [InputExample(texts=['My first sentence', 'My second sentence'], label=0.8),
   InputExample(texts=['Another pair', 'Unrelated sentence'], label=0.3)]
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

In [5]:
def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [6]:
examples_dict = load_obj("training_examples_5_sentences")

In [33]:
examples = []
for key, value in examples_dict.items():
    for sentence in value:
        examples.append((key, sentence))

In [16]:
sentences = [sentence for key, sentence in examples]

In [8]:
#Define the model. Either from scratch of by loading a pre-trained model
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [34]:
#Define your train examples. You need more than just two examples...
# train_examples = [InputExample(texts=['My first sentence', 'My second sentence'], label=0.8),
#     InputExample(texts=['Another pair', 'Unrelated sentence'], label=0.3)]
train_examples = [InputExample(texts=[value], label=key) for key, value in examples]

In [10]:
len(train_examples)

11228

In [25]:
embeddings = model.encode(sentences)


In [27]:
key_embeddings = []
for i in range(len(examples)):
    key_embeddings.append((examples[i][0], embeddings[i]))
    
    

In [31]:
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [32]:
save_obj(key_embeddings, "default_embeddings")

In [35]:
train_dataset = SentencesDataset(train_examples, model)

In [36]:
#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)
train_loss = losses.BatchAllTripletLoss(model=model)

In [42]:
#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=2, warmup_steps=100)

In [39]:
embeddings2 = model.encode(sentences, show_progress_bar=True)

In [40]:
key_embeddings2 = []
for i in range(len(examples)):
    key_embeddings2.append((examples[i][0], embeddings2[i]))
    

In [41]:
save_obj(key_embeddings2, "default_embeddings2")

In [43]:
embeddings3 = model.encode(sentences, show_progress_bar=True)

In [44]:
key_embeddings3 = []
for i in range(len(examples)):
    key_embeddings3.append((examples[i][0], embeddings3[i]))
    

In [45]:
save_obj(key_embeddings3, "embeddings_3epoch")

In [ ]:
from sentence_transformers import evaluation
sentences1 = ['This list contains the first column', 'With your sentences', 'You want your model to evaluate on']
sentences2 = ['Sentences contains the other column', 'The evaluator matches sentences1[i] with sentences2[i]', 'Compute the cosine similarity and compares it to scores[i]']
scores = [0.3, 0.6, 0.2]

evaluator = evaluation.EmbeddingSimilarityEvaluator(sentences1, sentences2, scores)

# ... Your other code to load training data

# model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100, evaluator=evaluator, evaluation_steps=500)